In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import datetime
from datetime import datetime, timedelta
#import plotly.graph_objs as go
import matplotlib.pyplot as plt
import csv
import seaborn as sns
%matplotlib inline
sns.set()
from PIL import Image

# Dengue dataset:

The dataset is just the filtered confirmed cases, using spark, of the original dataset (see SINAN_extracao_dengue)

In [20]:
df = pd.read_csv('/media/irybak/My Passport/Datalake/SINAN/Dengue/dengue_conf/dengue_conf_14_11_2019.csv',low_memory=False)

In [5]:
df.columns

Index(['tp_not', 'id_agravo', 'year_sin_pri', 'month_sin_pri', 'dt_notific',
       'sem_not', 'dt_sin_pri', 'sem_pri', 'classi_fin', 'criterio',
       'sg_uf_not', 'id_municip', 'cs_sexo', 'cs_gestant', 'cs_raca',
       'cs_escol_n', 'codmunres', 'nu_idade_n', 'comuninf', 'count'],
      dtype='object')

# Observe that the variable 'year_sin_pri' has several missprint. 

In [4]:
df.year_sin_pri.value_counts()

2015.0    1375488
2013.0    1158156
2016.0    1066258
2010.0     853372
2011.0     586694
2014.0     471402
2012.0     401926
2017.0     165398
2018.0     143823
2009.0       7387
2001.0        471
1994.0        298
1996.0        298
1997.0        294
1995.0        285
1982.0        277
1992.0        276
1989.0        275
1993.0        275
1981.0        264
1998.0        261
1991.0        260
1983.0        258
1986.0        257
1978.0        256
1990.0        252
1980.0        250
1987.0        249
1985.0        242
1988.0        239
           ...   
1934.0         14
1931.0         13
1932.0         13
1936.0         12
1929.0         11
1933.0          8
2106.0          6
1930.0          6
1928.0          6
1921.0          3
1925.0          3
1923.0          2
1201.0          2
201.0           2
1924.0          2
1926.0          2
1927.0          2
1922.0          1
1920.0          1
1918.0          1
1917.0          1
1915.0          1
1912.0          1
1911.0          1
1018.0    

## We will create year and month from  variable 'dt_notific' in order to use in our work. (see dengue_case_file for the problematic relating to the variable tp extract the week)

In [21]:
df['Mycol'] =  df['dt_notific'].apply(pd.to_datetime)

In [22]:
df['year'] = pd.DatetimeIndex(df['Mycol']).year
df['month'] = pd.DatetimeIndex(df['Mycol']).month

In [8]:
df.year.value_counts() # number of cases per year

2015    1366872
2013    1165706
2016    1081567
2010     856801
2011     589455
2014     471774
2012     397735
2017     166478
2018     145708
Name: year, dtype: int64

# Construct the dataset of dengue with the desired variables

In [23]:
data = df.filter(['codmunres','year','cs_sexo','cs_raca','nu_idade_n'])

## Process to clean age group and padronize following the age category given in population from IBGE

In [10]:
#data.nu_idade_n.unique()

In [24]:
vector =[]
for value in data.nu_idade_n.astype(str):
    if value == '1': value = 2001
    elif value == '2': value = 2001
    elif value == '3': value = 3001
    elif 4<= int(value) < 10: value = '400'+value
    elif len(value) == 2: value = '40' + value
    vector.extend([value])

In [25]:
#vector

In [26]:
data['nu_idade_n_cor'] = vector

In [27]:
data.columns

Index(['codmunres', 'year', 'cs_sexo', 'cs_raca', 'nu_idade_n',
       'nu_idade_n_cor'],
      dtype='object')

In [28]:
#data[data['nu_idade_n']==80]

In [ ]:
# Categorize age group
# 0,  101,  202,  303,  404,  505,  606,  707,  808,  909, 1010,
#       1111, 1212, 1313, 1414, 1515, 1616, 1717, 1818, 1919, 2024, 2529,
#       3034, 3539, 4044, 4549, 5054, 5559, 6064, 6569, 7074, 7579, 8099],

In [29]:
fx = []
for value in data.nu_idade_n_cor:
    if 1000 <= int(value) <4005: faixa = '4' 
    elif 4005 <=int(value)< 4010: faixa = '509'
    elif 4010 <=int(value)< 4015: faixa = '1014'
    elif 4015 <=int(value)< 4020: faixa = '1519'
    elif 4020 <=int(value)< 4025: faixa = '2024'
    elif 4025 <=int(value)< 4030: faixa = '2529'
    elif 4030 <=int(value)< 4035: faixa = '3034'
    elif 4035 <=int(value)< 4040: faixa = '3539' 
    elif 4040 <=int(value)< 4045: faixa = '4044'   
    elif 4045 <=int(value)< 4050: faixa = '4549'
    elif 4050 <=int(value)< 4055: faixa = '5054'
    elif 4055 <=int(value)< 4060: faixa = '5559'
    elif 4060 <=int(value)< 4065: faixa = '6064'
    elif 4065 <=int(value)< 4070: faixa = '6569'
    elif 4070 <=int(value)< 4075: faixa = '7074'
    elif 4075 <=int(value)< 4080: faixa = '7579'
    elif 4080 <=int(value)< 8100: faixa = '8099'
    fx.extend([faixa])             

In [30]:
data['faixa'] = fx
data

,codmunres,year,cs_sexo,cs_raca,nu_idade_n,nu_idade_n_cor,faixa
0,510340.0,2010,M,1.0,4040,4040,4044
1,510340.0,2010,F,4.0,4030,4030,3034
2,510340.0,2010,M,4.0,4021,4021,2024
3,314330.0,2010,F,1.0,4050,4050,5054
4,311940.0,2010,M,9.0,4013,4013,1014
...,...,...,...,...,...,...,...
6242091,172100.0,2018,M,4.0,4023,4023,2024
6242092,170210.0,2018,M,4.0,4032,4032,3034
6242093,170765.0,2018,F,4.0,4070,4070,7074
6242094,172100.0,2018,M,1.0,4007,4007,509


In [17]:
data.cs_sexo.value_counts()

F    3485260
M    2749309
I       7316
Name: cs_sexo, dtype: int64

In [31]:
data['count'] = 1

In [32]:
dados = data.groupby(['codmunres','year','cs_sexo','cs_raca','faixa'])['count'].sum().reset_index()

In [33]:
dados.cs_sexo = dados.cs_sexo.replace('F', 1)
dados.cs_sexo= dados.cs_sexo.replace('M', 2)

In [34]:
dados.head()

,codmunres,year,cs_sexo,cs_raca,faixa,count
0,110000.0,2015,1,1.0,4,1
1,110000.0,2016,2,9.0,6064,1
2,110001.0,2010,1,1.0,1014,2
3,110001.0,2010,1,1.0,2024,1
4,110001.0,2010,1,1.0,3539,1


In [22]:
dados.groupby(['year'])['count'].sum()

year
2010     856757
2011     589375
2012     397675
2013    1165564
2014     471700
2015    1366701
2016    1081538
2017     166474
2018     145707
Name: count, dtype: int64

In [23]:
df.year.value_counts()

2015    1366872
2013    1165706
2016    1081567
2010     856801
2011     589455
2014     471774
2012     397735
2017     166478
2018     145708
Name: year, dtype: int64

In [24]:
df.year_sin_pri.value_counts()

2015.0    1375488
2013.0    1158156
2016.0    1066258
2010.0     853372
2011.0     586694
           ...   
1016.0          1
1015.0          1
218.0           1
215.0           1
2916.0          1
Name: year_sin_pri, Length: 113, dtype: int64

# Population

In [5]:
#pop = pd.read_csv('G:\Datalake\POPULACAO\csv\POPBR12.csv')   #'G:\Datalake\POPULACAO\pop_race_sex_age.csv')

In [49]:
pop = pd.read_csv('/media/irybak/My Passport/Datalake/POPULACAO/estimativas_pop_sex_age_muni_cidacs/pop_2000_2017_SFA.csv')
pop = pop.drop(columns=['Unnamed: 0'])

In [50]:
pop.columns

Index(['MUNIC_RES', 'ANO', 'SEXO', 'FXETARIA', 'POPULACAO'], dtype='object')

In [51]:
pop.FXETARIA.unique()

array([   0,  101,  202,  303,  404,  505,  606,  707,  808,  909, 1010,
       1111, 1212, 1313, 1414, 1515, 1616, 1717, 1818, 1919, 2024, 2529,
       3034, 3539, 4044, 4549, 5054, 5559, 6064, 6569, 7074, 7579, 8099])

In [52]:
pop = pop.replace({'FXETARIA': {0: 4, 101:4, 202:4,  303:4,  404:4,
                                505: 509, 606: 509,  707: 509,  808: 509,  909: 509,
                                1010: 1014, 1111: 1014, 1212: 1014, 1313: 1014, 1414: 1014,
                                1515: 1519, 1616: 1519, 1717: 1519, 1818: 1519, 1919: 1519}})

In [53]:
pop.head()

,MUNIC_RES,ANO,SEXO,FXETARIA,POPULACAO
0,110001,2000,1,4,118
1,110001,2000,1,4,110
2,110001,2000,1,4,127
3,110001,2000,1,4,149
4,110001,2000,1,4,140


# Creating tables to map the standard incidence ratio of dengue per year

## expected number of cases per year and cummulated

In [31]:
teste = dados

In [32]:
teste = teste[(2010 <= teste.year) & (teste.year <= 2018) & (teste.cs_sexo != 'I')]

In [33]:
teste.cs_sexo.unique()

array([1, 2], dtype=object)

In [34]:
teste.year.unique()

array([2015, 2016, 2010, 2011, 2012, 2013, 2014, 2017, 2018])

In [35]:
teste['faixa'] = teste['faixa'].astype(int)

/home/irybak/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
teste.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 415402 entries, 0 to 417957
Data columns (total 5 columns):
codmunres    415402 non-null float64
year         415402 non-null int64
cs_sexo      415402 non-null object
faixa        415402 non-null int64
count        415402 non-null int64
dtypes: float64(1), int64(3), object(1)
memory usage: 19.0+ MB


In [37]:
teste.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 415402 entries, 0 to 417957
Data columns (total 5 columns):
codmunres    415402 non-null float64
year         415402 non-null int64
cs_sexo      415402 non-null object
faixa        415402 non-null int64
count        415402 non-null int64
dtypes: float64(1), int64(3), object(1)
memory usage: 19.0+ MB


In [38]:
sum_observed_gender_age_year = []
for year in [2010,2011,2012,2013,2014,2015,2016,2017,2018]:
    df_year = teste[teste.year == year]
    table_year = pd.pivot_table(df_year, values='count', index=['codmunres'], columns=['cs_sexo','faixa'])
    table_year=table_year.fillna(0)
    resulted_dataframe = table_year.sum().reset_index()
    sum_observed_gender_age_year.append(resulted_dataframe)

In [54]:
sum_observed_gender_age_year[8]

,cs_sexo,faixa,0
0,1,4,3629.0
1,1,509,4099.0
2,1,1014,5332.0
3,1,1519,7621.0
4,1,2024,8355.0
5,1,2529,7666.0
6,1,3034,7709.0
7,1,3539,7387.0
8,1,4044,6464.0
9,1,4549,5708.0


In [55]:
pop.head()

,MUNIC_RES,ANO,SEXO,FXETARIA,POPULACAO
0,110001,2000,1,4,118
1,110001,2000,1,4,110
2,110001,2000,1,4,127
3,110001,2000,1,4,149
4,110001,2000,1,4,140


# Population 

In [54]:
pop = pop.astype(int)

In [57]:
teste_pop = pop.groupby(['MUNIC_RES','ANO','SEXO','FXETARIA'])['POPULACAO'].sum().reset_index()
teste_pop.head()

,MUNIC_RES,ANO,SEXO,FXETARIA,POPULACAO
0,110001,2000,1,4,1496
1,110001,2000,1,509,1677
2,110001,2000,1,1014,1570
3,110001,2000,1,1519,1522
4,110001,2000,1,2024,1292


In [58]:
sum_pop_gender_age_year = []
for year in [2010,2011,2012,2013,2014,2015,2016,2017,2018]:
    df_year = teste_pop[teste_pop.ANO == year]
    table_year = pd.pivot_table(df_year, values='POPULACAO', index=['MUNIC_RES'], columns=['SEXO','FXETARIA'])
    table_year=table_year.fillna(0)
    resulted_dataframe = table_year.sum().reset_index()
    sum_pop_gender_age_year.append(resulted_dataframe)

In [67]:
sum_pop_gender_age_year[8]

,level_0,SEXO,FXETARIA,0


In [31]:
#table_pop = pd.pivot_table(teste_pop, values='POPULACAO', index=['MUNIC_RES'], columns=['SEXO','FXETARIA'])
#table_pop

In [32]:
#sum_pop_gender_age_year = table_pop.sum().reset_index()

In [33]:
#sum_pop_gender_age_year.columns

In [34]:
#sum_pop_gender_age_year.rename(columns={0:'pop'}, 
#                 inplace=True)

In [35]:
#sum_pop_gender_age_year.head()

In [36]:
#sum_pop_gender_age_year['pop']

In [43]:
sum_pop_gender_age_year[0]

,SEXO,FXETARIA,0
0,1,4,7016987
1,1,509,7624144
2,1,1014,8725413
3,1,1519,8558868
4,1,2024,8630227
5,1,2529,8460995
6,1,3034,7717657
7,1,3539,6766665
8,1,4044,6320570
9,1,4549,5692013


# calculating r_ j (for each year)

In [86]:
#r_j = []
#for df in sum_observed_gender_age_year:
#    risco_relativo = df[0]/sum_pop_gender_age_year['pop']
#    r_j.append(risco_relativo)

In [68]:
r_j = []
for df1,df2 in zip(sum_observed_gender_age_year,sum_pop_gender_age_year):
    #df2.rename(columns={0:'pop'}, inplace=True)
    risco_relativo = df1[0]/df2[0]
    r_j.append(risco_relativo)

In [69]:
r_j[3]

0     0.002603
1     0.003187
2     0.005233
3     0.007989
4     0.008689
5     0.008032
6     0.007928
7     0.007998
8     0.008124
9     0.007863
10    0.007414
11    0.006993
12    0.005983
13    0.005467
14    0.004755
15    0.003762
16    0.002571
17    0.002596
18    0.003127
19    0.005164
20    0.006855
21    0.006984
22    0.005981
23    0.005499
24    0.005304
25    0.005284
26    0.004937
27    0.004445
28    0.004183
29    0.003816
30    0.003646
31    0.003261
32    0.002758
33    0.001924
Name: 0, dtype: float64

# calculating E_i for each municipality

In [70]:
table_pop = pd.pivot_table(teste_pop, values='POPULACAO', index=['MUNIC_RES'], columns=['SEXO','FXETARIA'])
table_pop

SEXO                   1                                               \
FXETARIA            4              509            1014           1519   
MUNIC_RES                                                               
110001       1215.555556    1302.222222    1328.277778    1353.944444   
110002       4295.444444    4377.611111    4564.666667    4488.055556   
110003        293.888889     315.055556     351.833333     344.500000   
110004       3455.055556    3532.166667    3831.888889    3970.111111   
110005        743.666667     787.555556     870.333333     863.111111   
...                  ...            ...            ...            ...   
522200        538.333333     538.055556     569.166667     576.055556   
522205        288.111111     301.888889     300.222222     319.722222   
522220        219.555556     230.055556     224.388889     214.500000   
522230        212.333333     242.111111     264.833333     241.166667   
530010     106350.333333  103601.388889  108832.333333  116362.722222   

SEXO                                                                   \
FXETARIA            2024           2529           3034           3539   
MUNIC_RES                                                               
110001       1192.555556    1107.722222    1020.222222     959.833333   
110002       4179.444444    3960.666667    3703.611111    3284.500000   
110003        298.777778     282.111111     258.611111     237.111111   
110004       3691.111111    3422.888889    3215.611111    2946.944444   
110005        797.500000     687.500000     671.222222     614.000000   
...                  ...            ...            ...            ...   
522200        571.055556     541.277778     556.611111     513.166667   
522205        323.444444     316.944444     294.222222     277.611111   
522220        199.333333     217.722222     178.500000     146.333333   
522230        220.833333     187.222222     206.222222     173.222222   
530010     118290.888889  117489.388889  115017.666667  102369.722222   

SEXO                                   ...              2                \
FXETARIA           4044          4549  ...           3539          4044   
MUNIC_RES                              ...                                
110001       854.222222    715.444444  ...     915.611111    738.222222   
110002      2918.444444   2346.000000  ...    3218.055556   2769.500000   
110003       221.111111    200.555556  ...     216.666667    199.444444   
110004      2625.666667   2187.111111  ...    2954.222222   2623.833333   
110005       567.388889    496.111111  ...     621.555556    554.666667   
...                 ...           ...  ...            ...           ...   
522200       455.388889    374.833333  ...     490.388889    429.833333   
522205       241.388889    211.222222  ...     254.055556    233.333333   
522220       120.944444    108.111111  ...     136.277778    115.500000   
522230       169.611111    166.722222  ...     154.777778    144.500000   
530010     85628.555556  70874.944444  ...  111346.055556  93335.666667   

SEXO                                                               \
FXETARIA           4549          5054          5559          6064   
MUNIC_RES                                                           
110001       625.055556    474.111111    392.055556    290.888889   
110002      2223.777778   1667.000000   1280.055556    926.611111   
110003       189.666667    156.944444    128.222222    108.444444   
110004      2133.777778   1696.555556   1333.611111    979.000000   
110005       466.500000    381.388889    300.388889    248.111111   
...                 ...           ...           ...           ...   
522200       355.111111    284.444444    236.722222    194.277778   
522205       190.055556    140.611111    129.555556     89.388889   
522220        87.555556     72.277778     62.666667     33.333333   
522230       142.277778    106.333333    104.500000     76.166667   
530010

In [71]:
teste_pop.head()

,MUNIC_RES,ANO,SEXO,FXETARIA,POPULACAO
0,110001,2000,1,4,1496
1,110001,2000,1,509,1677
2,110001,2000,1,1014,1570
3,110001,2000,1,1519,1522
4,110001,2000,1,2024,1292


In [123]:
teste.head()

,codmunres,year,cs_sexo,faixa,count
0,110000.0,2015,1,4,1
1,110000.0,2016,2,6064,1
2,110001.0,2010,1,1014,2
3,110001.0,2010,1,2024,1
4,110001.0,2010,1,3539,1


# 2010

In [72]:
filter_2010 = teste_pop[teste_pop.ANO == 2010]

In [73]:
P_2010 = pd.pivot_table(filter_2010, values='POPULACAO', index=['MUNIC_RES'], columns=['SEXO','FXETARIA'])
P_2010;

In [124]:
P_2010.head()

SEXO          1                                                        ...  \
FXETARIA   4     509   1014  1519  2024  2529  3034  3539  4044  4549  ...   
MUNIC_RES                                                              ...   
110001      956  1073  1256  1362  1105  1053   951   878   913   846  ...   
110002     3645  4177  4740  4479  4456  4161  3755  3431  3291  2613  ...   
110003      240   253   313   317   226   250   237   242   247   213  ...   
110004     2958  3203  3756  3976  3796  3450  3170  2938  2815  2436  ...   
110005      626   700   841   875   765   681   639   587   639   581  ...   

SEXO          2                                                    
FXETARIA   3539  4044  4549  5054  5559  6064 6569 7074 7579 8099  
MUNIC_RES                                                          
110001      917   802   720   515   431   311  264  177  119   73  
110002     3513  3125  2429  1828  1484  1041  735  497  317  294  
110003      215   220   202   173   142    94   65   68   34   26  
110004     3129  2832  2396  1945  1427  1008  795  607  431  380  
110005      662   629   548   406   311   301  196  134   97   84  

[5 rows x 34 columns]

In [74]:
E_2010 =[]
for i in range(len(P_2010)):
    E_i = sum(np.array(P_2010.iloc[i])*np.array(r_j[0]))
    E_2010.append([P_2010.index.values[i],E_i])

In [75]:
Expected_2010 = pd.DataFrame(E_2010,
                 columns=['codmunres', 'expected_2010'])

In [76]:
Expected_2010.head()

,codmunres,expected_2010
0,110001,111.128432
1,110002,411.105553
2,110003,28.592163
3,110004,357.275068
4,110005,77.181413


# 2011

In [77]:
filter_2011 = teste_pop[teste_pop.ANO == 2011]

In [78]:
P_2011 = pd.pivot_table(filter_2011, values='POPULACAO', index=['MUNIC_RES'], columns=['SEXO','FXETARIA'])
P_2011;

In [79]:
E_2011 =[]
for i in range(len(P_2011)):
    E_i = sum(np.array(P_2011.iloc[i])*np.array(r_j[1]))
    E_2011.append([P_2011.index.values[i],E_i])

In [80]:
Expected_2011 = pd.DataFrame(E_2011,
                 columns=['codmunres', 'expected_2011'])

In [81]:
Expected_2011.head()

,codmunres,expected_2011
0,110001,75.629524
1,110002,286.775292
2,110003,19.216999
3,110004,245.546874
4,110005,52.430493


# 2012

In [82]:
filter_2012 = teste_pop[teste_pop.ANO == 2012]

In [83]:
P_2012 = pd.pivot_table(filter_2012, values='POPULACAO', index=['MUNIC_RES'], columns=['SEXO','FXETARIA'])
P_2012;

In [84]:
E_2012 =[]
for i in range(len(P_2012)):
    E_i = sum(np.array(P_2012.iloc[i])*np.array(r_j[2]))
    E_2012.append([P_2012.index.values[i],E_i])

In [85]:
Expected_2012 = pd.DataFrame(E_2012,
                 columns=['codmunres', 'expected_2012'])

In [86]:
Expected_2012.head()

,codmunres,expected_2012
0,110001,50.215073
1,110002,194.174986
2,110003,12.639273
3,110004,165.491926
4,110005,34.884951


# 2013

In [87]:
filter_2013 = teste_pop[teste_pop.ANO == 2013]

In [88]:
P_2013 = pd.pivot_table(filter_2013, values='POPULACAO', index=['MUNIC_RES'], columns=['SEXO','FXETARIA'])
P_2013;

In [89]:
E_2013 =[]
for i in range(len(P_2013)):
    E_i = sum(np.array(P_2013.iloc[i])*np.array(r_j[3]))
    E_2013.append([P_2013.index.values[i],E_i])

In [90]:
Expected_2013 = pd.DataFrame(E_2013,
                 columns=['codmunres', 'expected_2013'])

In [91]:
Expected_2013.head()

,codmunres,expected_2013
0,110001,130.005645
1,110002,537.160010
2,110003,34.810636
3,110004,471.714230
4,110005,95.667674


# 2014

In [92]:
filter_2014 = teste_pop[teste_pop.ANO == 2014]

In [93]:
P_2014 = pd.pivot_table(filter_2014, values='POPULACAO', index=['MUNIC_RES'], columns=['SEXO','FXETARIA'])
P_2014;

In [94]:
E_2014 =[]
for i in range(len(P_2014)):
    E_i = sum(np.array(P_2014.iloc[i])*np.array(r_j[4]))
    E_2014.append([P_2014.index.values[i],E_i])

In [95]:
Expected_2014 = pd.DataFrame(E_2014,
                 columns=['codmunres', 'expected_2014'])

In [96]:
Expected_2014.head()

,codmunres,expected_2014
0,110001,51.266263
1,110002,217.612998
2,110003,13.755320
3,110004,190.044238
4,110005,37.800425


# 2015

In [97]:
filter_2015 = teste_pop[teste_pop.ANO == 2015]

In [98]:
P_2015 = pd.pivot_table(filter_2015, values='POPULACAO', index=['MUNIC_RES'], columns=['SEXO','FXETARIA'])
P_2015;

In [99]:
E_2015 =[]
for i in range(len(P_2015)):
    E_i = sum(np.array(P_2015.iloc[i])*np.array(r_j[5]))
    E_2015.append([P_2015.index.values[i],E_i])

In [100]:
Expected_2015 = pd.DataFrame(E_2015,
                 columns=['codmunres', 'expected_2015'])

In [101]:
Expected_2015.head()

,codmunres,expected_2015
0,110001,145.464884
1,110002,646.498188
2,110003,37.536266
3,110004,544.199523
4,110005,109.095933


# 2016

In [102]:
filter_2016 = teste_pop[teste_pop.ANO == 2016]

In [103]:
P_2016 = pd.pivot_table(filter_2016, values='POPULACAO', index=['MUNIC_RES'], columns=['SEXO','FXETARIA'])
P_2016;

In [104]:
E_2016 =[]
for i in range(len(P_2016)):
    E_i = sum(np.array(P_2016.iloc[i])*np.array(r_j[6]))
    E_2016.append([P_2016.index.values[i],E_i])

In [105]:
Expected_2016 = pd.DataFrame(E_2016,
                 columns=['codmunres', 'expected_2016'])

In [106]:
Expected_2016.head()

,codmunres,expected_2016
0,110001,112.210692
1,110002,515.501707
2,110003,28.626501
3,110004,429.302577
4,110005,84.608077


# 2017

In [107]:
filter_2017 = teste_pop[teste_pop.ANO == 2017]

In [108]:
P_2017 = pd.pivot_table(filter_2017, values='POPULACAO', index=['MUNIC_RES'], columns=['SEXO','FXETARIA'])
P_2017;

In [109]:
E_2017 =[]
for i in range(len(P_2017)):
    E_i = sum(np.array(P_2017.iloc[i])*np.array(r_j[7]))
    E_2017.append([P_2017.index.values[i],E_i])

In [110]:
Expected_2017 = pd.DataFrame(E_2017,
                 columns=['codmunres', 'expected_2017'])

In [111]:
Expected_2017.head()

,codmunres,expected_2017
0,110001,16.924089
1,110002,81.589610
2,110003,4.303680
3,110004,66.362854
4,110005,12.835925


# 2018

In [112]:
filter_2018 = teste_pop[teste_pop.ANO == 2018]

In [113]:
P_2018 = pd.pivot_table(filter_2018, values='POPULACAO', index=['MUNIC_RES'], columns=['SEXO','FXETARIA'])
P_2018;

In [114]:
sum_observed_gender_age_year[8]

,cs_sexo,faixa,0
0,1,4,3629.0
1,1,509,4099.0
2,1,1014,5332.0
3,1,1519,7621.0
4,1,2024,8355.0
5,1,2529,7666.0
6,1,3034,7709.0
7,1,3539,7387.0
8,1,4044,6464.0
9,1,4549,5708.0


In [115]:
r_j[8] =  sum_observed_gender_age_year[8][0]/sum_pop_gender_age_year[0][0]

In [116]:
E_2018 =[]
for i in range(len(P_2010)):
    E_i = sum(np.array(P_2010.iloc[i])*np.array(r_j[8]))
    E_2018.append([P_2010.index.values[i],E_i])

In [117]:
Expected_2018 = pd.DataFrame(E_2018,
                 columns=['codmunres', 'expected_2018'])

In [118]:
Expected_2018.head()

,codmunres,expected_2018
0,110001,18.815249
1,110002,69.451243
2,110003,4.864562
3,110004,60.518160
4,110005,13.089248


# Observed number of cases per municipality and year

In [127]:
observed = df.groupby(['codmunres','year'])['count'].sum().reset_index()
observed.head()

,codmunres,year,count
0,110000.0,2015,1
1,110000.0,2016,1
2,110001.0,2010,13
3,110001.0,2011,1
4,110001.0,2012,2


In [128]:
observed = observed[(2010 <= observed.year) & (observed.year <= 2018)]

In [129]:
observed = pd.pivot_table(observed, values='count', index=['codmunres'], columns=['year'])

In [130]:
observed = observed.fillna(0) 
observed.head()

year,2010,2011,2012,2013,2014,2015,2016,2017,2018
codmunres,,,,,,,,,
110000.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
110001.0,13.0,1.0,2.0,159.0,5.0,10.0,12.0,10.0,3.0
110002.0,228.0,27.0,2.0,35.0,56.0,21.0,50.0,61.0,7.0
110003.0,61.0,13.0,42.0,35.0,3.0,92.0,43.0,5.0,3.0
110004.0,939.0,31.0,18.0,442.0,31.0,46.0,217.0,25.0,13.0


# calculate the standard incidence ratio

In [131]:
codigo_muni = pd.read_excel('/media/irybak/My Passport/Datalake/POPULACAO/População/POP_python/DTB_BRASIL_MUNICIPIO.xls')

In [132]:
codigo_muni.head()

,UF,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Código Município Completo,Nome_Município
0,11,Rondônia,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste
1,11,Rondônia,2,Leste Rondoniense,6,Cacoal,379,1100379,Alto Alegre dos Parecis
2,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,403,1100403,Alto Paraíso
3,11,Rondônia,2,Leste Rondoniense,5,Alvorada D'Oeste,346,1100346,Alvorada D'Oeste
4,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,23,1100023,Ariquemes


In [133]:
codigo_muni.columns

Index(['UF', 'Nome_UF', 'Mesorregião Geográfica', 'Nome_Mesorregião',
       'Microrregião Geográfica', 'Nome_Microrregião', 'Município',
       'Código Município Completo', 'Nome_Município'],
      dtype='object')

In [134]:
codigo_muni.rename(columns={'Código Município Completo':'codmuni_full'}, 
                 inplace=True);

In [135]:
codigo_muni['codmuni'] = codigo_muni.codmuni_full.floordiv(10)

In [136]:
codigo_muni.head()

,UF,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,codmuni_full,Nome_Município,codmuni
0,11,Rondônia,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste,110001
1,11,Rondônia,2,Leste Rondoniense,6,Cacoal,379,1100379,Alto Alegre dos Parecis,110037
2,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,403,1100403,Alto Paraíso,110040
3,11,Rondônia,2,Leste Rondoniense,5,Alvorada D'Oeste,346,1100346,Alvorada D'Oeste,110034
4,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,23,1100023,Ariquemes,110002


In [137]:
observed.columns

Int64Index([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018], dtype='int64', name='year')

In [138]:
final = codigo_muni.set_index('codmuni').join(observed) 

In [139]:
final

,UF,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,codmuni_full,Nome_Município,2010,2011,2012,2013,2014,2015,2016,2017,2018
codmuni,,,,,,,,,,,,,,,,,,
110001,11,Rondônia,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste,13.0,1.0,2.0,159.0,5.0,10.0,12.0,10.0,3.0
110037,11,Rondônia,2,Leste Rondoniense,6,Cacoal,379,1100379,Alto Alegre dos Parecis,27.0,0.0,5.0,113.0,4.0,9.0,6.0,6.0,2.0
110040,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,403,1100403,Alto Paraíso,33.0,35.0,36.0,57.0,18.0,3.0,5.0,22.0,4.0
110034,11,Rondônia,2,Leste Rondoniense,5,Alvorada D'Oeste,346,1100346,Alvorada D'Oeste,70.0,276.0,84.0,515.0,13.0,34.0,82.0,6.0,1.0
110002,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,23,1100023,Ariquemes,228.0,27.0,2.0,35.0,56.0,21.0,50.0,61.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522200,52,Goiás,5,Sul Goiano,16,Pires do Rio,22005,5222005,Vianópolis,40.0,37.0,8.0,28.0,111.0,86.0,114.0,36.0,13.0
522205,52,Goiás,5,Sul Goiano,15,Meia Ponte,22054,5222054,Vicentinópolis,39.0,7.0,43.0,114.0,26.0,113.0,36.0,3.0,85.0
522220,52,Goiás,4,Leste Goiano,12,Entorno de Brasília,22203,5222203,Vila Boa,1.0,5.0,0.0,409.0,225.0,5.0,2.0,26.0,0.0


In [140]:
final = final.join(Expected_2018.set_index('codmunres')).join(Expected_2017.set_index('codmunres')).join(Expected_2016.set_index('codmunres')).join(Expected_2015.set_index('codmunres')).join(Expected_2014.set_index('codmunres')).join(Expected_2013.set_index('codmunres')).join(Expected_2012.set_index('codmunres')).join(Expected_2011.set_index('codmunres')).join(Expected_2010.set_index('codmunres'))

In [141]:
final.columns

Index([                     'UF',                 'Nome_UF',
        'Mesorregião Geográfica',        'Nome_Mesorregião',
       'Microrregião Geográfica',       'Nome_Microrregião',
                     'Município',            'codmuni_full',
                'Nome_Município',                      2010,
                            2011,                      2012,
                            2013,                      2014,
                            2015,                      2016,
                            2017,                      2018,
                 'expected_2018',           'expected_2017',
                 'expected_2016',           'expected_2015',
                 'expected_2014',           'expected_2013',
                 'expected_2012',           'expected_2011',
                 'expected_2010'],
      dtype='object')

In [142]:
final['SIR_2010'] = final[2010]/final['expected_2010']
final['SIR_2011'] = final[2011]/final['expected_2011']
final['SIR_2012'] = final[2012]/final['expected_2012']
final['SIR_2013'] = final[2013]/final['expected_2013']
final['SIR_2014'] = final[2014]/final['expected_2014']
final['SIR_2015'] = final[2015]/final['expected_2015']
final['SIR_2016'] = final[2016]/final['expected_2016']
final['SIR_2017'] = final[2017]/final['expected_2017']
final['SIR_2018'] = final[2018]/final['expected_2018']

In [143]:
final

,UF,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,codmuni_full,Nome_Município,2010,...,expected_2010,SIR_2010,SIR_2011,SIR_2012,SIR_2013,SIR_2014,SIR_2015,SIR_2016,SIR_2017,SIR_2018
codmuni,,,,,,,,,,,,,,,,,,,,,
110001,11,Rondônia,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste,13.0,...,111.128432,0.116982,0.013222,0.039829,1.223024,0.097530,0.068745,0.106942,0.590874,0.159445
110037,11,Rondônia,2,Leste Rondoniense,6,Cacoal,379,1100379,Alto Alegre dos Parecis,27.0,...,58.131264,0.464466,0.000000,0.187101,1.673373,0.147867,0.109932,0.093292,0.598126,0.203336
110040,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,403,1100403,Alto Paraíso,33.0,...,77.814982,0.424083,0.640327,0.974591,0.565209,0.437142,0.025101,0.052222,1.454469,0.303841
110034,11,Rondônia,2,Leste Rondoniense,5,Alvorada D'Oeste,346,1100346,Alvorada D'Oeste,70.0,...,76.397016,0.916266,5.339912,2.470414,5.635528,0.361361,0.331782,1.043450,0.506245,0.077286
110002,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,23,1100023,Ariquemes,228.0,...,411.105553,0.554602,0.094150,0.010300,0.065157,0.257338,0.032483,0.096993,0.747644,0.100790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522200,52,Goiás,5,Sul Goiano,16,Pires do Rio,22005,5222005,Vianópolis,40.0,...,56.832975,0.703817,0.949536,0.303777,0.365589,3.587338,0.970657,1.623599,3.315710,1.344109
522205,52,Goiás,5,Sul Goiano,15,Meia Ponte,22054,5222054,Vicentinópolis,39.0,...,33.777557,1.154613,0.300159,2.700373,2.965150,1.682338,2.329105,0.933435,0.501834,14.805258
522220,52,Goiás,4,Leste Goiano,12,Entorno de Brasília,22203,5222203,Vila Boa,1.0,...,21.693166,0.046097,0.326066,0.000000,14.222027,19.297038,0.154573,0.077524,6.463668,0.000000


In [145]:
final.to_excel('standard_incidence_ratio_dengue.xls')

cs_sexo -> 'F'= 1

cs_sexo -> 'M'= 2

cs_raca -> 1- branca

           2- preta
           
           3- amarela
           
           4- parda
           
           5- indígena
           
           9 Ignorado
           
           
cor -> Branca- 1

        Preta- 2 
        
        Amarela- 3
        
        Parda- 4 
        
        Indigena- 5 
        
        Sem declaraçao- 9
                  
Sexo -> 'Mulheres' = 1

        'Homens' = 2 
       
Idade -> '0 a 4 anos': 4, 

         '5 a 9 anos': 509,
         
         '10 a 14 anos': 1014,
         
         '15 a 19 anos': 1519,
         
         '20 a 24 anos': 2024, 
         
         '25 a 29 anos': 2529, 
         
         '30 a 34 anos': 3034,
         
         '35 a 39 anos': 3539,
         
         '40 a 44 anos': 4044, 
         
         '45 a 49 anos': 4549,
         
         '50 a 54 anos': 5054, 
         
         '55 a 59 anos': 5559,
         
         '60 a 64 anos': 6064, 
         
         '65 a 69 anos': 6569, 
         
         '70 a 74 anos': 7074, 
         
         '75 a 79 anos': 7579,
         
         '80 a 89 anos': 8099, 
         
         '90 a 99 anos': 8099 

In [35]:
df['cs_raca'] = df['cs_raca'].fillna(0)
df['cs_raca'] = df['cs_raca'].astype(float).astype(int)

In [41]:
df = dados[(2010 <= dados.year) & (dados.year <= 2018) & (dados.cs_sexo != 'I')]

In [45]:
df.head()

,codmunres,year,cs_sexo,cs_raca,faixa,count
0,110000,2015,1,1,4,1
1,110000,2016,2,9,6064,1
2,110001,2010,1,1,1014,2
3,110001,2010,1,1,2024,1
4,110001,2010,1,1,3539,1


In [44]:
df = df.astype(int)

In [46]:
key = df.agg('{0[codmunres]}{0[cs_sexo]}{0[cs_raca]}{0[faixa]}'.format, axis=1)
df['key'] = key

In [47]:
df.head()

,codmunres,year,cs_sexo,cs_raca,faixa,count,key
0,110000,2015,1,1,4,1,110000114
1,110000,2016,2,9,6064,1,110000296064
2,110001,2010,1,1,1014,2,110001111014
3,110001,2010,1,1,2024,1,110001112024
4,110001,2010,1,1,3539,1,110001113539


In [55]:
pop.head()

,MUNIC_RES,ANO,SEXO,FXETARIA,POPULACAO
0,110001,2000,1,4,118
1,110001,2000,1,4,110
2,110001,2000,1,4,127
3,110001,2000,1,4,149
4,110001,2000,1,4,140


In [10]:
pop['codmun'] = pop['codmun'].astype(str).str[:-1].astype(np.int64)
pop.head()

,Unnamed: 0,codmun,mame_muni,cor,Sexo,Idade,Total
0,0,110001,Alta Floresta D'Oeste (RO),1,2,4,430
1,1,110001,Alta Floresta D'Oeste (RO),1,2,509,393
2,2,110001,Alta Floresta D'Oeste (RO),1,2,1014,484
3,3,110001,Alta Floresta D'Oeste (RO),1,2,1519,524
4,4,110001,Alta Floresta D'Oeste (RO),1,2,2024,462


In [11]:
y = pop.agg('{0[codmun]}{0[Sexo]}{0[cor]}{0[Idade]}'.format, axis=1)

In [12]:
pop['key1'] = y

In [65]:
pop

,codmun,mame_muni,cor,Sexo,Idade,Total,key1
0,110001,Alta Floresta D'Oeste (RO),1,2,4,430,110001214
1,110001,Alta Floresta D'Oeste (RO),1,2,509,393,11000121509
2,110001,Alta Floresta D'Oeste (RO),1,2,1014,484,110001211014
3,110001,Alta Floresta D'Oeste (RO),1,2,1519,524,110001211519
4,110001,Alta Floresta D'Oeste (RO),1,2,2024,462,110001212024
5,110001,Alta Floresta D'Oeste (RO),1,2,2529,444,110001212529
6,110001,Alta Floresta D'Oeste (RO),1,2,3034,387,110001213034
7,110001,Alta Floresta D'Oeste (RO),1,2,3539,359,110001213539
8,110001,Alta Floresta D'Oeste (RO),1,2,4044,383,110001214044
9,110001,Alta Floresta D'Oeste (RO),1,2,4549,363,110001214549


In [14]:
df.year.unique()

array([2015, 2010, 2011, 2012, 2013, 2014, 2017, 2018, 2016, 2009, 2045,
       2083, 2084, 2067, 2077, 2066, 2095, 2090, 2089], dtype=int64)

In [15]:
pop = pop.drop(columns=['Unnamed: 0'])

In [16]:
data_year = []
lst = [2015, 2010, 2011, 2012, 2013, 2014, 2017, 2018, 2016, 2009]
for value in lst:
    data = df[df['year'] == value]
    data_year.append(data)

# Table of number of cases per gender, race, age-group and year in Brazil

In [ ]:
teste = df.groupby(['cs_sexo','cs_raca','faixa','year'])['count'].sum().reset_index()
teste.head()

In [ ]:
table1 = pd.pivot_table(teste, values='count', index=['cs_sexo','cs_raca','faixa'], columns=['year'])

In [ ]:
table1.to_excel('table1.xls')

# Table with population per gender, race, age-group and year in Brazil

In [88]:
teste2 = pop.groupby(['Sexo','cor','Idade'])['Total'].sum().reset_index()

In [92]:
teste2.to_excel('table2.xls')

In [49]:
def filter_data(dataframe,value_year,value_gender,value_race):
    data = dataframe[(dataframe.year == value_year) &(dataframe.cs_sexo == value_gender) & (dataframe.cs_raca==value_race)]
    return data

In [52]:
data_female_white_2010 = filter_data(df,2010,'1',1)

In [63]:
data_female_white_2010.head()

,codmunres,year,cs_sexo,cs_raca,faixa,count,key
1,110001,2010,1,1,1014,2,110001111014
2,110001,2010,1,1,2024,1,110001112024
3,110001,2010,1,1,3539,1,110001113539
4,110001,2010,1,1,4549,1,110001114549
5,110001,2010,1,1,5054,1,110001115054


In [56]:
data_female_white_2010.groupby(['year','faixa'])['count'].sum().reset_index()

,year,faixa,count
0,2010,4,5833
1,2010,509,7572
2,2010,1014,12969
3,2010,1519,16780
4,2010,2024,18018
5,2010,2529,18152
6,2010,3034,17085
7,2010,3539,15602
8,2010,4044,14859
9,2010,4549,13813


,cs_sexo,cs_raca,faixa,year,count
0,1,1,4,2009,15
1,1,1,4,2010,5833
2,1,1,4,2011,3612
3,1,1,4,2012,1872
4,1,1,4,2013,5131


In [17]:
result = []
for data in data_year:
    df1 =pop.set_index('key1').join(data.set_index('key')) 
    result.append(df1)

In [18]:
result[0]

,codmun,mame_muni,cor,Sexo,Idade,Total,codmunres,year,cs_sexo,cs_raca,faixa,count
110001111014,110001,Alta Floresta D'Oeste (RO),1,1,1014,427,NaN,NaN,NaN,NaN,NaN,NaN
110001111519,110001,Alta Floresta D'Oeste (RO),1,1,1519,503,NaN,NaN,NaN,NaN,NaN,NaN
110001112024,110001,Alta Floresta D'Oeste (RO),1,1,2024,445,NaN,NaN,NaN,NaN,NaN,NaN
110001112529,110001,Alta Floresta D'Oeste (RO),1,1,2529,444,110001.0,2015.0,1,1.0,2529.0,1.0
110001113034,110001,Alta Floresta D'Oeste (RO),1,1,3034,362,NaN,NaN,NaN,NaN,NaN,NaN
110001113539,110001,Alta Floresta D'Oeste (RO),1,1,3539,369,NaN,NaN,NaN,NaN,NaN,NaN
110001114,110001,Alta Floresta D'Oeste (RO),1,1,4,446,110001.0,2015.0,1,1.0,4.0,1.0
110001114044,110001,Alta Floresta D'Oeste (RO),1,1,4044,366,NaN,NaN,NaN,NaN,NaN,NaN
110001114549,110001,Alta Floresta D'Oeste (RO),1,1,4549,342,NaN,NaN,NaN,NaN,NaN,NaN
110001115054,110001,Alta Floresta D'Oeste (RO),1,1,5054,234,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
result[0].cs_raca.unique()
result[0].cor.unique()

array([1, 2, 3, 4, 5, 9], dtype=int64)

In [21]:
result[0][result[0].cs_raca ==1]

,codmun,mame_muni,cor,Sexo,Idade,Total,codmunres,year,cs_sexo,cs_raca,faixa,count
110001112529,110001,Alta Floresta D'Oeste (RO),1,1,2529,444,110001.0,2015.0,1,1.0,2529.0,1.0
110001114,110001,Alta Floresta D'Oeste (RO),1,1,4,446,110001.0,2015.0,1,1.0,4.0,1.0
110001212024,110001,Alta Floresta D'Oeste (RO),1,2,2024,462,110001.0,2015.0,2,1.0,2024.0,1.0
110001213034,110001,Alta Floresta D'Oeste (RO),1,2,3034,387,110001.0,2015.0,2,1.0,3034.0,1.0
110002111014,110002,Ariquemes (RO),1,1,1014,1455,110002.0,2015.0,1,1.0,1014.0,1.0
110002114549,110002,Ariquemes (RO),1,1,4549,1030,110002.0,2015.0,1,1.0,4549.0,1.0
110002116064,110002,Ariquemes (RO),1,1,6064,470,110002.0,2015.0,1,1.0,6064.0,1.0
110003111519,110003,Cabixi (RO),1,1,1519,126,110003.0,2015.0,1,1.0,1519.0,2.0
110003112024,110003,Cabixi (RO),1,1,2024,92,110003.0,2015.0,1,1.0,2024.0,2.0
110003112529,110003,Cabixi (RO),1,1,2529,113,110003.0,2015.0,1,1.0,2529.0,1.0


cs_sexo -> 'F'= 1
cs_sexo -> 'M'= 2

cs_raca -> 1- branca
           2- preta
           3- amarela
           4- parda
           5- indígena
           9 Ignorado
           
           
cor -> Branca- 1
        Preta- 2 
        Amarela- 3
        Parda- 4 
        Indigena- 5 
        Sem declaraçao- 9
                  
Sexo -> 'Mulheres' = 1
        'Homens' = 2 
       
Idade -> '0 a 4 anos': 4, 
         '5 a 9 anos': 509,
         '10 a 14 anos': 1014,
         '15 a 19 anos': 1519,
         '20 a 24 anos': 2024, 
         '25 a 29 anos': 2529, 
         '30 a 34 anos': 3034, 
         '35 a 39 anos': 3539,
         '40 a 44 anos': 4044, 
         '45 a 49 anos': 4549,
         '50 a 54 anos': 5054, 
         '55 a 59 anos': 5559,
         '60 a 64 anos': 6064, 
         '65 a 69 anos': 6569, 
         '70 a 74 anos': 7074, 
         '75 a 79 anos': 7579,
         '80 a 89 anos': 8099, 
         '90 a 99 anos': 8099 